<a href="https://colab.research.google.com/github/DPotoyan/Chem167/blob/master/Hoomd_Colab_LJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title #### Installs 

import sys
print(sys.version)
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
!conda config --set always_yes yes
!conda config --add channels conda-forge
!conda config --add channels conda-forge/label/hoomd_dev
!conda create -n hoomd python=3.7 git hoomd gsd 
sys.path.append('/usr/local/envs/hoomd/lib/python3.7/site-packages')

3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
--2021-10-27 19:30:20--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66709754 (64M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  63.62M   142MB/s    in 0.4s    

2021-10-27 19:30:21 (142 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [66709754/66709754]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py39h27cfd23_1003
    - ca-certificates==2021.7.5=h06a4308_1
    - certifi==2021.5.30=py39h06a4308_0
    -

In [17]:
import itertools
import math
import hoomd, gsd.hoomd

import numpy as np
import pandas as pd
import plotly.express as px

### Create LJ fluid

In [6]:
m = 4
N_particles = 4 * m**3

spacing = 1.3
K = math.ceil(N_particles**(1 / 3))
L = K * spacing
x = numpy.linspace(-L / 2, L / 2, K, endpoint=False)

position = list(itertools.product(x, repeat=3))

In [7]:
# Config
snapshot                    = gsd.hoomd.Snapshot()
snapshot.configuration.box  = [L, L, L, 0, 0, 0]

# Particles
snapshot.particles.N        = N_particles
snapshot.particles.position = position[0:N_particles]
snapshot.particles.types    = ['A']
snapshot.particles.typeid   = [0] * N_particles

with gsd.hoomd.open(name='lattice.gsd', mode='xb') as f:
    f.append(snapshot)

### Run MD

In [14]:
#Add pair potentials to non-bonded pairs
cell = hoomd.md.nlist.Cell()
lj = hoomd.md.pair.LJ(nlist=cell)

lj.params[('A', 'A')] = dict(epsilon=1, sigma=1)
lj.r_cut[('A', 'A')] = 2.5

# Create integrator, add forces and assign ensemble
integrator = hoomd.md.Integrator(dt=0.005)
integrator.forces.append(lj)

nvt = hoomd.md.methods.NVT(kT=1.5, filter=hoomd.filter.All(), tau=1.0)
integrator.methods.append(nvt)

# Create sim
device  = hoomd.device.CPU() # CPU or GPU
sim = hoomd.Simulation(device=device, seed=123) 
sim.operations.integrator = integrator

# Load gsd 
sim.create_state_from_gsd(filename='lattice.gsd')

# Set velocities
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=1.5)

# output dcd or gsd
gsd_writer = hoomd.write.GSD(filename='traj.gsd',
                             trigger=hoomd.trigger.Periodic(1000))

sim.operations.writers.append(gsd_writer)

sim.run(1e5)

### Visualize

In [18]:
import ipywidgets
from ipywidgets import interact

traj = gsd.hoomd.open('traj.gsd')

@interact(i=(0, len(traj)-1))
def render_frame(i=0):

  color={0:'blue', 1: 'red'}
  df = pd.DataFrame()

  p = traj[i].particles

  df['x']     = p.position[:, 0]
  df['y']     = p.position[:, 1]
  df['z']     = p.position[:, 2]
  df['color'] = np.array([color[i] for i in p.typeid])

  return px.scatter_3d(df, x='x', y='y', z='z', color='color', 
                       range_x = [df.min(), df.max()], 
                       range_y = [df.min(), df.max()], 
                       range_z = [df.min(), df.max()],
                       width=800, height=800)

interactive(children=(IntSlider(value=0, description='i', max=99), Output()), _dom_classes=('widget-interact',…